### getdata

In [ ]:
from tvDatafeed import TvDatafeed, Interval
import pandas as pd

# หากไม่มีบัญชี TradingView
tv = TvDatafeed()

# ดึงข้อมูลราคาหุ้น TSLA จาก NASDAQ ในกรอบเวลา 1 ชั่วโมง (1H)
data = tv.get_hist(symbol='BTCUSDT', exchange='BINANCE', interval=Interval.in_daily, n_bars=5000)
# data = tv.get_hist(symbol='SPX', exchange='TVC', interval=Interval.in_daily, n_bars=5000)
# data = tv.get_hist(symbol='AMD', exchange='NASDAQ', interval=Interval.in_15_minute, n_bars=5000)

# ตรวจสอบว่ามีข้อมูลหรือไม่
if data is not None and not data.empty:
    # แปลง index เป็นคอลัมน์ Date
    data.reset_index(inplace=True)
    data.rename(columns={'datetime': 'Date', 'close': 'Close', 'high': 'High', 'low': 'Low'}, inplace=True)  # เปลี่ยนชื่อเป็น Date
    
    # บันทึกข้อมูลลงไฟล์ CSV
    filename = "BTC.csv"
    data.to_csv(filename, index=False)  # index=False เพราะเราเก็บ Date ไว้แล้ว
    print(f"Data saved to {filename}")
else:
    print("No data retrieved")


In [ ]:
import MetaTrader5 as mt5
import pandas as pd
from datetime import datetime, timedelta

# ตั้งค่าชื่อสินทรัพย์และกรอบเวลา
symbol = "BTCUSD"
timeframe = mt5.TIMEFRAME_D1 # กรอบเวลา 1 วัน

# กำหนดช่วงวันที่
start_date = datetime(2000, 1, 1)  # วันที่ 5 ก.พ. 2024
end_date = datetime(2025, 2, 28)  # วันที่ 18 ก.พ. 2025

# เริ่มต้นการเชื่อมต่อกับ MT5
if not mt5.initialize():
    print("Failed to connect to MT5")
    exit()

# ดึงข้อมูลราคาตามช่วงวันที่
rates = mt5.copy_rates_range(symbol, timeframe, start_date, end_date)

# ปิดการเชื่อมต่อกับ MT5
mt5.shutdown()

# แปลงข้อมูลเป็น DataFrame
if rates is not None and len(rates) > 0:
    df = pd.DataFrame(rates).rename(columns={'time': 'Date', 'close': 'close', 'high': 'high', 'low': 'low', 'open': 'open','tick_volume':'volume'})
    
    # แปลง timestamp เป็น datetime และเพิ่มเวลาไทย (UTC+7)
    df['Date'] = pd.to_datetime(df['Date'], unit='s') 

    # ตั้งค่า Date เป็น index เพื่อให้เป็น DatetimeIndex
    df.set_index('Date', inplace=True)

    # บันทึกลงไฟล์ CSV
    filename = f"BTC_MT5_1D.csv"
    df.to_csv(filename)
    
    print(f"Data saved to {filename}")
else:
    print("No data retrieved")


In [ ]:
import yfinance as yf
import pandas as pd

# กำหนดช่วงเวลาที่ต้องการดึงข้อมูล
start_date = "2015-01-01"
end_date = "2025-03-10"

# ฟังก์ชันดาวน์โหลดข้อมูลและบันทึก CSV
def download_data(ticker, filename):
    data = yf.download(ticker, start=start_date, end=end_date)
    
    # เลือก 'Adj Close' ถ้ามี หรือใช้ 'Close' แทน
    price_column = "Adj Close" if "Adj Close" in data.columns else "Close"
    
    if price_column in data.columns:
        # ใช้ Index เป็น Date และเปลี่ยนชื่อคอลัมน์เป็น "Price"
        data = data[[price_column]].rename(columns={price_column: "Price"})
        
        # บันทึก CSV โดยไม่มี header ซ้อนกัน
        data.to_csv(filename, index=True, index_label="Date", header=True)
        print(f"✅ {ticker} data saved to {filename}")
    else:
        print(f"❌ Warning: {ticker} data does not contain 'Adj Close' or 'Close'.")

# ดึงข้อมูล Bitcoin (BTC-USD)
download_data("BTC-USD", "btc.csv")

# ดึงข้อมูล S&P 500 (^GSPC)
download_data("^GSPC", "sp500.csv")


In [50]:
import yfinance as yf
import pandas as pd

# ดึงข้อมูลหุ้น DELTA.BK
data = yf.download("BTC-USD", start='2015-01-01', end='2025-03-10', interval='1d')

# ถ้ามี MultiIndex Headers ให้ลดเหลือชั้นเดียว
if isinstance(data.columns, pd.MultiIndex):
    data.columns = data.columns.get_level_values(0)  # ดึงเฉพาะชั้นแรก เช่น Open, High, Low, Close, Volume

# รีเซ็ต index เพื่อให้ Date เป็นคอลัมน์
data = data.reset_index()

# เปลี่ยนชื่อคอลัมน์ให้ถูกต้อง
data.rename(columns={'Date': 'Date', 'Open': 'Open', 'High': 'High', 'Low': 'Low', 'Close': 'Close', 'Adj Close': 'Adj Close', 'Volume': 'Volume'}, inplace=True)

# แสดงตัวอย่างข้อมูล
print(data.head())
data.to_csv("btc.csv")


[*********************100%***********************]  1 of 1 completed

Price       Date       Close        High         Low        Open    Volume
0     2015-01-01  314.248993  320.434998  314.002991  320.434998   8036550
1     2015-01-02  315.032013  315.838989  313.565002  314.079010   7860650
2     2015-01-03  281.082001  315.149994  281.082001  314.846008  33054400
3     2015-01-04  264.195007  287.230011  257.612000  281.145996  55629100
4     2015-01-05  274.473999  278.341003  265.084015  265.084015  43962800


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf

# Download Bitcoin and S&P 500 data
btc = yf.download("BTC-USD", start="2015-01-01", end="2025-3-10")
sp500 = yf.download("^GSPC", start="2015-01-01", end="2025-3-10")

# Check available columns
print("BTC Columns:", btc.columns)
print("S&P 500 Columns:", sp500.columns)

# Use 'Adj Close' if available, otherwise use 'Close'
btc_column = "Adj Close" if "Adj Close" in btc.columns else "Close"
sp500_column = "Adj Close" if "Adj Close" in sp500.columns else "Close"

# Ensure both datasets have the necessary column
btc = btc[[btc_column]].rename(columns={btc_column: "BTC"})
sp500 = sp500[[sp500_column]].rename(columns={sp500_column: "SP500"})

# Merge data by date
merged = btc.join(sp500, how="inner")

# Calculate log returns
merged["BTC_Return"] = np.log(merged["BTC"] / merged["BTC"].shift(1))
merged["SP500_Return"] = np.log(merged["SP500"] / merged["SP500"].shift(1))

# Compute Rolling 1-Year Correlation (252 trading days)
merged["Rolling_Corr"] = merged["BTC_Return"].rolling(window=252).corr(merged["SP500_Return"])

# Plot the Rolling Correlation
plt.figure(figsize=(12, 6))
plt.plot(merged["Rolling_Corr"], color='blue', label='BTC vs S&P 500')
plt.fill_between(merged.index, merged["Rolling_Corr"], where=(merged["Rolling_Corr"] < 0), color='red', alpha=0.5)

# Customize plot
plt.axhline(0, color='black', linestyle='--', linewidth=0.8)
plt.title("Bitcoin and S&P 500: Rolling 1-Year Correlation Coefficient")
plt.xlabel("Year")
plt.ylabel("Correlation")
plt.grid(True)
plt.legend()
plt.show()


In [ ]:
merged

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf

# Download Bitcoin and S&P 500 data
btc = yf.download("BTC-USD", start="2012-01-01", end="2022-12-31")
sp500 = yf.download("^GSPC", start="2012-01-01", end="2022-12-31")

# Use 'Adj Close' if available, otherwise use 'Close'
btc_column = "Adj Close" if "Adj Close" in btc.columns else "Close"
sp500_column = "Adj Close" if "Adj Close" in sp500.columns else "Close"

# Ensure both datasets have the necessary column
btc = btc[[btc_column]].rename(columns={btc_column: "BTC"})
sp500 = sp500[[sp500_column]].rename(columns={sp500_column: "SP500"})

# Merge data by date
merged = btc.join(sp500, how="inner")

# Calculate log returns
merged["BTC_Return"] = np.log(merged["BTC"] / merged["BTC"].shift(1))
merged["SP500_Return"] = np.log(merged["SP500"] / merged["SP500"].shift(1))

# Compute Rolling 1-Year Correlation (252 trading days)
merged["Rolling_Corr"] = merged["BTC_Return"].rolling(window=252).corr(merged["SP500_Return"])

# Smooth the correlation line to match the original chart style
merged["Rolling_Corr_Smoothed"] = merged["Rolling_Corr"].rolling(window=90).mean()

# Plot the Rolling Correlation
plt.figure(figsize=(12, 6))
plt.plot(merged.index, merged["Rolling_Corr_Smoothed"], color='blue', label='BTC vs S&P 500', linewidth=2)
plt.fill_between(merged.index, merged["Rolling_Corr_Smoothed"], where=(merged["Rolling_Corr_Smoothed"] < 0), color='red', alpha=0.6)

# Customize plot
plt.axhline(0, color='black', linestyle='--', linewidth=1)
plt.ylim(merged["Rolling_Corr_Smoothed"].min() - 0.05, merged["Rolling_Corr_Smoothed"].max() + 0.05)  # Auto adjust Y-axis
plt.title("Bitcoin and S&P 500: Rolling 1-Year Correlation Coefficient")
plt.xlabel("Year")
plt.ylabel("Correlation")
plt.grid(True, linestyle='dotted', alpha=0.7)
plt.legend()
plt.show()

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf

# Download Bitcoin and S&P 500 data
btc = yf.download("BTC-USD", start="2012-01-01", end="2022-12-31")
sp500 = yf.download("^GSPC", start="2012-01-01", end="2022-12-31")

# Use 'Adj Close' if available, otherwise use 'Close'
btc_column = "Adj Close" if "Adj Close" in btc.columns else "Close"
sp500_column = "Adj Close" if "Adj Close" in sp500.columns else "Close"

# Ensure both datasets have the necessary column
btc = btc[[btc_column]].rename(columns={btc_column: "BTC"})
sp500 = sp500[[sp500_column]].rename(columns={sp500_column: "SP500"})

# Merge data by date
merged = btc.join(sp500, how="inner")

# Calculate log returns
merged["BTC_Return"] = np.log(merged["BTC"] / merged["BTC"].shift(1))
merged["SP500_Return"] = np.log(merged["SP500"] / merged["SP500"].shift(1))

# Compute Rolling 1-Year Correlation (252 trading days)
merged["Rolling_Corr"] = merged["BTC_Return"].rolling(window=252).corr(merged["SP500_Return"])

# Smooth the correlation line to match the original chart style
merged["Rolling_Corr_Smoothed"] = merged["Rolling_Corr"].rolling(window=90).mean()

# Drop NaN values to ensure the first valid correlation value appears
merged = merged.dropna()

# Plot the Rolling Correlation
plt.figure(figsize=(12, 6))
plt.plot(merged.index, merged["Rolling_Corr_Smoothed"], color='blue', label='BTC vs S&P 500', linewidth=2)
plt.fill_between(merged.index, merged["Rolling_Corr_Smoothed"], where=(merged["Rolling_Corr_Smoothed"] < 0), color='red', alpha=0.6)

# Customize plot
plt.axhline(0, color='black', linestyle='--', linewidth=1)
plt.ylim(merged["Rolling_Corr_Smoothed"].min() - 0.05, merged["Rolling_Corr_Smoothed"].max() + 0.05)  # Auto adjust Y-axis
plt.xlim(merged.index.min(), merged.index.max())  # Ensure X-axis starts at earliest date
plt.title("Bitcoin and S&P 500: Rolling 1-Year Correlation Coefficient")
plt.xlabel("Year")
plt.ylabel("Correlation")
plt.grid(True, linestyle='dotted', alpha=0.7)
plt.legend()
plt.show()


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf

# Download Bitcoin and S&P 500 data
btc = yf.download("BTC-USD", start="2015-01-01", end="2022-12-31")
sp500 = yf.download("^GSPC", start="2015-01-01", end="2022-12-31")

# Use 'Adj Close' if available, otherwise use 'Close'
btc_column = "Adj Close" if "Adj Close" in btc.columns else "Close"
sp500_column = "Adj Close" if "Adj Close" in sp500.columns else "Close"

# Ensure both datasets have the necessary column
btc = btc[[btc_column]].rename(columns={btc_column: "BTC"})
sp500 = sp500[[sp500_column]].rename(columns={sp500_column: "SP500"})

# Merge data by date
merged = btc.join(sp500, how="inner")

# Calculate log returns
merged["BTC_Return"] = np.log(merged["BTC"] / merged["BTC"].shift(1))
merged["SP500_Return"] = np.log(merged["SP500"] / merged["SP500"].shift(1))

# Compute Rolling 1-Year Correlation (252 trading days)
merged["Rolling_Corr"] = merged["BTC_Return"].rolling(window=252).corr(merged["SP500_Return"])

# Smooth the correlation line to match the original chart style
merged["Rolling_Corr_Smoothed"] = merged["Rolling_Corr"].rolling(window=90).mean()

# Drop NaN values to ensure the first valid correlation value appears
merged = merged.dropna()

# Plot the Rolling Correlation
plt.figure(figsize=(12, 6))
plt.plot(merged.index, merged["Rolling_Corr_Smoothed"], color='blue', label='BTC vs S&P 500', linewidth=2)
plt.fill_between(merged.index, merged["Rolling_Corr_Smoothed"], where=(merged["Rolling_Corr_Smoothed"] < 0), color='red', alpha=0.6)

# Customize plot
plt.axhline(0, color='black', linestyle='--', linewidth=1)
plt.ylim(merged["Rolling_Corr_Smoothed"].min() - 0.05, merged["Rolling_Corr_Smoothed"].max() + 0.05)  # Auto adjust Y-axis
plt.xlim(merged.index.min(), merged.index.max())  # Ensure X-axis starts at earliest date
plt.title("Bitcoin and S&P 500: Rolling 1-Year Correlation Coefficient")
plt.xlabel("Year")
plt.ylabel("Correlation")
plt.grid(True, linestyle='dotted', alpha=0.7)
plt.legend()
plt.show()


In [ ]:
merged